In [46]:
import requests
import json
import pandas as pd
from dateutil import parser
import matplotlib
import matplotlib.pyplot as plt


KEY = "secret_dhDJCxIGksOzhxiKis5UfH8VWA6JXTnF5T44BANarrG"
headers = {"Authorization":KEY, "Notion-Version":"2022-06-28", "Content-Type":"application/json"}
base_url = "https://api.notion.com/v1/databases/"
publication_database_id = '54cb3960b40246e2be42cfb58e2b45fd'
people_database_id = '778b56a297cc4fd7b45d8f40c3d4e1cb'
query_url = base_url+people_database_id+"/query"
print("query_url:", query_url)
#filter_rule = {"or": [dict(property="年月", "")]}
filter_rule={}
sort_rules = [dict(property="graduation", direction="ascending")]

query_data = dict(page_size=100, sorts=sort_rules)#, filter=filter_rule)

#res = requests.post(query_url, headers=headers, query_data=query_data)
#result = res.content
#obj = json.loads( result.decode() )
#df = pd.DataFrame(obj["results"])


query_url: https://api.notion.com/v1/databases/778b56a297cc4fd7b45d8f40c3d4e1cb/query


In [47]:
import copy
def fetch_notion_db(query_url, headers, query_data):
    query_data = copy.deepcopy(query_data)
    result_list = []
    counter = 0
    #query_data["start_cursor"] = "undefined"
    while True:
        print(counter)
        res = json.loads( requests.post(query_url, headers=headers, data=json.dumps(query_data)).content.decode() )
        #print(res)
        has_more = res.get("has_more")
        result_list = result_list + res["results"]
        print(len(result_list))
        print(has_more, res["next_cursor"], query_data.get("start_cursor", None))
        if has_more is None or has_more==False:
            break
        query_data["start_cursor"] = res["next_cursor"]
        #query_data["start_cursor"] = res["next_cursor"]
        counter+=1
    return result_list

datalist = fetch_notion_db(query_url, headers, query_data)
df = pd.DataFrame(datalist)

0
{'object': 'list', 'results': [{'object': 'page', 'id': '0626a8a7-1eae-456a-81d3-e9b6644405b4', 'created_time': '2023-06-03T05:17:00.000Z', 'last_edited_time': '2023-06-03T05:17:00.000Z', 'created_by': {'object': 'user', 'id': '147bdf86-4261-4d70-ba95-2923623aa778'}, 'last_edited_by': {'object': 'user', 'id': '147bdf86-4261-4d70-ba95-2923623aa778'}, 'cover': None, 'icon': None, 'parent': {'type': 'database_id', 'database_id': '778b56a2-97cc-4fd7-b45d-8f40c3d4e1cb'}, 'archived': False, 'properties': {'alumni_next': {'id': '%3CoIx', 'type': 'rich_text', 'rich_text': []}, 'Photo': {'id': 'GJSM', 'type': 'files', 'files': []}, 'email': {'id': 'Kof%40', 'type': 'email', 'email': None}, 'graduation': {'id': '_JgX', 'type': 'number', 'number': 0}, 'photo_name': {'id': 'fAJp', 'type': 'rich_text', 'rich_text': [{'type': 'text', 'text': {'content': 'jonliu.jpg', 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, 'color': '

In [48]:
datalist

[{'object': 'page',
  'id': '0626a8a7-1eae-456a-81d3-e9b6644405b4',
  'created_time': '2023-06-03T05:17:00.000Z',
  'last_edited_time': '2023-06-03T05:17:00.000Z',
  'created_by': {'object': 'user',
   'id': '147bdf86-4261-4d70-ba95-2923623aa778'},
  'last_edited_by': {'object': 'user',
   'id': '147bdf86-4261-4d70-ba95-2923623aa778'},
  'cover': None,
  'icon': None,
  'parent': {'type': 'database_id',
   'database_id': '778b56a2-97cc-4fd7-b45d-8f40c3d4e1cb'},
  'archived': False,
  'properties': {'alumni_next': {'id': '%3CoIx',
    'type': 'rich_text',
    'rich_text': []},
   'Photo': {'id': 'GJSM', 'type': 'files', 'files': []},
   'email': {'id': 'Kof%40', 'type': 'email', 'email': None},
   'graduation': {'id': '_JgX', 'type': 'number', 'number': 0},
   'photo_name': {'id': 'fAJp',
    'type': 'rich_text',
    'rich_text': [{'type': 'text',
      'text': {'content': 'jonliu.jpg', 'link': None},
      'annotations': {'bold': False,
       'italic': False,
       'strikethrough': F

In [41]:
down.keys(), down["next_cursor"]

(dict_keys(['object', 'results', 'next_cursor', 'has_more', 'type', 'page']),
 'd6d478a7-e3e7-4c80-a17b-18945abc7194')

In [37]:
len(down["results"]), down["results"][1]["properties"]

(100,
 {'alumni_next': {'id': '%3CoIx', 'type': 'rich_text', 'rich_text': []},
  'Photo': {'id': 'GJSM', 'type': 'files', 'files': []},
  'email': {'id': 'Kof%40', 'type': 'email', 'email': None},
  'graduation': {'id': '_JgX', 'type': 'number', 'number': 0},
  'photo_name': {'id': 'fAJp',
   'type': 'rich_text',
   'rich_text': [{'type': 'text',
     'text': {'content': 'hui_huang.png', 'link': None},
     'annotations': {'bold': False,
      'italic': False,
      'strikethrough': False,
      'underline': False,
      'code': False,
      'color': 'default'},
     'plain_text': 'hui_huang.png',
     'href': None}]},
  'role': {'id': 'jZaP',
   'type': 'multi_select',
   'multi_select': [{'id': 'h|{t',
     'name': 'Affiliated Faculty',
     'color': 'red'}]},
  'number_educ': {'id': 'r%3D%7DC', 'type': 'number', 'number': 0},
  'website': {'id': '%7CQhQ',
   'type': 'url',
   'url': 'http://vcc.szu.edu.cn/~huihuang/'},
  'Alumni': {'id': '~%7Clu', 'type': 'checkbox', 'checkbox': Fal